# ESIEE Paris — Data Engineering I — Assignment 2
> Author : Badr TAJINI

**Academic year:** 2025–2026  
**Program:** Data & Applications - Engineering - (FD)   
**Course:** Data Engineering I  

---


## Data inputs
Define your input paths. Use small CSV/JSON/Parquet files so the notebook runs locally. If your dataset requires credentials, create a **sample subset** and commit only that.

**Paths to set:**
- `SOURCE_A_PATH` (fact‑like dataset)
- `SOURCE_B_PATH` (dimension‑like dataset)
- `OUTPUT_BASE` (directory for Parquet output)


In [ ]:
# Read carefully the helper to review what is missing here
# Example (edit):
# SOURCE_A_PATH = 'data/source_a.csv'
# SOURCE_B_PATH = 'data/source_b.csv'
# OUTPUT_BASE   = 'out/a2_v3_output'
pass


## Pipeline API (implementations required)
Implement the following functions. Keep signatures stable. Use explicit schemas when possible. Log counts at each stage.


In [ ]:
from typing import Optional, Tuple
from pyspark.sql import DataFrame

def ingest(spark, path_a: str, path_b: str) -> Tuple[DataFrame, DataFrame]:
    """Load SOURCE_A and SOURCE_B. Apply explicit schemas where possible.
    Return two DataFrames with uniform column naming.
    """
    # write some code here
    raise NotImplementedError

def transform(df_a: DataFrame, df_b: DataFrame) -> DataFrame:
    """Clean, deduplicate, and normalize. Add parsed timestamps.
    Drop obvious null records. Prepare keys for join.
    """
    # write some code here
    raise NotImplementedError

def join_and_aggregate(df: DataFrame, dim: DataFrame) -> DataFrame:
    """Join with dim table. Handle potential skew (hint: salting or AQE).
    Compute business aggregates with window or groupBy.
    """
    # write some code here
    raise NotImplementedError

def write_out(df: DataFrame, base: str, partitions: list[str]) -> None:
    """Write Parquet, overwrite mode, partitioned by `partitions`.
    Optimize small files if needed (coalesce).
    """
    # write some code here
    raise NotImplementedError


## Tasks
1. **Ingest**: read `SOURCE_A_PATH`, `SOURCE_B_PATH`. Provide explicit schemas. Count rows and malformed records.
2. **Transform**: standardize column names, cast types, parse timestamps into UTC, deduplicate using keys.
3. **Join + Aggregate**: explain your join strategy. Mitigate skew. Produce a tidy table with daily metrics.
4. **Store**: write partitioned Parquet to `OUTPUT_BASE`, e.g., partition by `date` and one categorical column.
5. **Explain plans**: capture `df.explain(mode='formatted')` for transform, join, and final write.
6. **Quality gates**: implement three checks (row count non‑zero, null rate thresholds, referential coverage). Abort if a gate fails.
7. **Reproducibility**: document your Spark config and any seeds. Describe how to re‑run.


In [1]:
# Orchestration
## Replace raises with your implementation, then run this driver.
if spark is not None:
    # write some code here
    # df_a, df_b = ingest(spark, SOURCE_A_PATH, SOURCE_B_PATH)
    # stg = transform(df_a, df_b)
    # out = join_and_aggregate(stg, df_b)
    # print('Final count:', out.count())
    # print('Plan:')
    # out.explain(mode='formatted')
    pass


NameError: name 'spark' is not defined

# Assignment 2: ETL


## 1. Querying the Operational Database

Let's run a query to verify that the operational database has been properly restored and that we can issue a query to PostgreSQL:

In [4]:
#First cell won't run
!PGPORT=5433 psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT COUNT(DISTINCT user_id) AS number_users FROM retail.user;"

#Solution 1
!sudo -u postgres env PGPORT=5433 psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT COUNT(DISTINCT user_id) AS number_users FROM retail.user;"

# Solution 2
# 1- Run this in your terminal to create a read-only user for the database :
# sudo -u postgres env PGPORT=5433 psql -d esiee_full -v ON_ERROR_STOP=1 -c \
# "CREATE ROLE esiee_reader LOGIN PASSWORD 'azerty123';
# GRANT CONNECT ON DATABASE esiee_full TO esiee_reader;
# GRANT USAGE ON SCHEMA retail TO esiee_reader;
# GRANT SELECT ON ALL TABLES IN SCHEMA retail TO esiee_reader;
# ALTER DEFAULT PRIVILEGES IN SCHEMA retail GRANT SELECT ON TABLES TO esiee_reader;"
# 2- Then, in your notebook, set the environment variables like this:
# import os
# os.environ['PGHOST'] = '127.0.0.1'   # force TCP (not Unix socket)
# os.environ['PGPORT'] = '5433'
# os.environ['PGUSER'] = 'esiee_reader'
# os.environ['PGPASSWORD'] = 'azerty123'
# 3- then
# !psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT COUNT(DISTINCT user_id) AS number_users FROM retail.user;"

/bin/bash: line 1: psql: command not found
sudo: unknown user postgres
sudo: error initializing audit plugin sudoers_audit


**The correct answer should be 3022290.**

If running the cell above gives you the same answer, the everything should be in order.

If you're getting an error, fix it before moving on.

As a warmup exercise, write SQL queries against the operational database to answer the following questions and report the answers.
Place both your SQL queries and answers in the following cell, replacing the placeholder texts that exist there.
Each question needs to be answered by a _single_ SQL query (that is, it is not acceptable to run multiple SQL queries and then compute the answer yourself).

1. For `session_id` `789d3699-028e-4367-b515-b82e2cb5225f`, what was the purchase price?
2. How many products are sold by the brand "sokolov"?
3. What is the average purchase price of items purchased from the brand "febest"?
4. What is average number of events per user? (Report answer to two digits after the decimal point, i.e., XX.XX)

**write some code here**

// qcell_1b76x2 (keep this id for tracking purposes)

**Q1 SQL:**

SELECT....

**Q1 answer:**

!psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT price \
FROM retail.events \
WHERE session_id = '789d3699-028e-4367-b515-b82e2cb5225f' \
AND event_type = 'purchase';"

**Q2 SQL:**

SELECT....

**Q2 answer:**

XXXX

**Q3 SQL:**

SELECT....

**Q3 answer:**

XXXX

**Q4 SQL:**

SELECT....

**Q4 answer:**

XXXX


In [ ]:
# Q1 answer :

!psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT price \
FROM retail.events \
WHERE session_id = '789d3699-028e-4367-b515-b82e2cb5225f' \
AND event_type = 'purchase';"

# or

!psql "esiee_full" -v ON_ERROR_STOP=1 -c "SELECT * \
FROM retail.events \
WHERE session_id = '789d3699-028e-4367-b515-b82e2cb5225f' \
AND event_type = 'purchase';"

## 2. Setup

The following cell contains setup to measure wall clock time and memory usage. (Don't worry about the details, just run the cell)

In [ ]:
!pip install -U numpy pandas pyarrow matplotlib scipy
import sys, subprocess
try:
    import psutil  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "psutil"])
print("psutil is installed.")


from IPython.core.magic import register_cell_magic
import time, os, platform

# Try to import optional modules
try:
    import psutil
except Exception:
    psutil = None

try:
    import resource  # not available on Windows
except Exception:
    resource = None


def _rss_bytes():
    """Resident Set Size in bytes (cross-platform via psutil if available)."""
    if psutil is not None:
        return psutil.Process(os.getpid()).memory_info().rss
    # Fallback: unknown RSS → 0 
    return 0


def _peak_bytes():
    """
    Best-effort peak memory in bytes.
    - Windows: psutil peak working set (peak_wset)
    - Linux:   resource.ru_maxrss (KB → bytes)
    - macOS:   resource.ru_maxrss (bytes)
    Fallback to current RSS if unavailable.
    """
    sysname = platform.system()

    # Windows path: use psutil peak_wset if present
    if sysname == "Windows" and psutil is not None:
        mi = psutil.Process(os.getpid()).memory_info()
        peak = getattr(mi, "peak_wset", None)  # should be available on Windows
        if peak is not None:
            return int(peak)
        return int(mi.rss)

    # POSIX path: resource may be available
    if resource is not None:
        try:
            ru = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
            # On Linux ru_maxrss is in kilobytes; on macOS/BSD it is bytes
            if sysname == "Linux":
                return int(ru) * 1024
            else:
                return int(ru)
        except Exception:
            pass

    # Last resort
    return _rss_bytes()


@register_cell_magic
def timemem(line, cell):
    """
    Measure wall time and memory around the execution of this cell.

        %%timemem
        <your code>

    Notes:
    - RSS = resident memory after the cell.
    - Peak is OS-dependent (see _peak_bytes docstring).
    """
    ip = get_ipython()

    rss_before  = _rss_bytes()
    peak_before = _peak_bytes()
    t0 = time.perf_counter()

    # Execute the cell body
    result = ip.run_cell(cell)

    t1 = time.perf_counter()
    rss_after  = _rss_bytes()
    peak_after = _peak_bytes()

    wall = t1 - t0
    rss_delta_mb  = (rss_after  - rss_before)  / (1024 * 1024)
    peak_delta_mb = (peak_after - peak_before) / (1024 * 1024)

    print("======================================")
    print(f"Wall time: {wall:.3f} s")
    print(f"RSS Δ: {rss_delta_mb:+.2f} MB")
    print(f"Peak memory Δ: {peak_delta_mb:+.2f} MB (OS-dependent)")
    print("======================================")

    return result

## 3. The "Extract" in ETL

The operational database comprises the tables described in the helper.

For the "Extract" in ETL, we're going to extract the following CSV files, each corresponding to a table in the operational database:

- **user.csv**: `user_id, gender, birthdate`
- **session.csv**: `session_id, user_id`
- **product.csv**: `product_id, brand, category, product_name`
- **product_name.csv**: `category, product_name, description`
- **events.csv**: `event_time, event_type, session_id, product_id, price`
- **category.csv**: `category, description`
- **brand.csv**: `brand, description`

From these files, we'll build a data warehouse organized in a standard star schema that has the following tables:

- Dimension tables: `dim_user`, `dim_age`, `dim_brand`, `dim_category`, `dim_product`, `dim_date`, `dim_session`
- The main fact table `fact_events` with foreign keys: `date_key, user_key, age_key, product_key, brand_key, category_key, session_key`


Let's specify a "base directory":

In [ ]:
# Change to path on your local machine.
BASE_DIR = "/Users/btajini/de1-work/assignment2"

These are the commands that perform the "extraction":

In [ ]:
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."user"         TO '\''{BASE_DIR}/user.csv'\''         WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."session"      TO '\''{BASE_DIR}/session.csv'\''      WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."category"     TO '\''{BASE_DIR}/category.csv'\''     WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."brand"        TO '\''{BASE_DIR}/brand.csv'\''        WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."product_name" TO '\''{BASE_DIR}/product_name.csv'\'' WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."product"      TO '\''{BASE_DIR}/product.csv'\''      WITH (FORMAT csv, HEADER true)'
!psql cs451 -v ON_ERROR_STOP=1 -c '\copy "retail"."events"       TO '\''{BASE_DIR}/events.csv'\''       WITH (FORMAT csv, HEADER true)'

(Note that the quote style above will _not_ work for Windows machines. Please adjust accordingly.)

After the extraction, you should have 7 CSV files, each corresponding to a table in the operational database.

The CSV files should be stored in `BASE_DIR`.

The following code snippet should "just work" to initialize Spark.

In [ ]:
import findspark, os

# Change to path on your local machine.
os.environ["SPARK_HOME"] = "/Users/jimmylin/Dropbox/workspace/teaching/spark-4.0.0-bin-hadoop3"
findspark.init()

import os
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.functions import col

py = sys.executable  # the Python of this notebook (e.g., .../envs/yourenv/bin/python)
os.environ["PYSPARK_DRIVER_PYTHON"] = py
os.environ["PYSPARK_PYTHON"] = py

spark = SparkSession.getActiveSession() or (
    SparkSession.builder
    .appName("A2")
    .master("local[*]")
    .config("spark.driver.memory", "8g")           # or 12g+
    .config("spark.sql.shuffle.partitions","400")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.pyspark.driver.python", py)
    .config("spark.pyspark.python", py)
    .config("spark.executorEnv.PYSPARK_PYTHON", py)
    .getOrCreate()
)

spark

At this point, Spark should be initialized.

Let's then load in CSV files into DataFrames.

write some code here

In [ ]:
%%timemem
# codecell_30z8le (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, we've loaded each of the CSV files into a corresponding dataframe.
# Let's count the number of records in each:

print(f"user: {df_user.count()}")
print(f"session: {df_session.count()}")
print(f"product: {df_product.count()}")
print(f"product_name: {df_product_name.count()}")
print(f"events: {df_events.count()}")
print(f"category: {df_category.count()}")
print(f"brand: {df_brand.count()}")

How do you know if you've done everything correctly?

Well, issue the SQL query `select count(*) from retail.user;` to count the number of rows in the `user` table in the operational database.
It should match the output of `df_user.count()`; same for the other tables.
If the counts match, then you know everything is in order.

## 4. Build the Dimensions Tables

### 4.1 The `user` Dimension Table

Build the `dim_user` dimension table.
This table should include `user_key`, `user_id`, `gender`, `birthdate`, and `generation`. 

Set `generation` to one of the following values based on the birth year: 
- "Traditionalists": born 1925 to 1945
- "Boomers": born 1946 to 1964
- "GenX": born 1965 to 1980
- "Millennials": born 1981 to 2000
- "GenZ": born 2001 to 2020

**write some code here**

In [1]:
%%timemem
# codecell_41ax14 (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "dim_user" should hold the user dimensions table according to the specification above.

dim_user.count()

UsageError: Cell magic `%%timemem` not found.


**The correct answer should be 3022290.**

### 4.2 The `age` Dimension Table

Even though `birthdate` exists in `dim_user`, a separate `dim_age` is helpful because it:
- Simplifies analysis with ready-made bands.
- Ensures consistency across all queries.
- Improves performance via small surrogate keys.
- Preserves history by fixing age at event time.
- Adds flexibility to adjust bands without changing facts.

We're going to build a `dim_age` table that has 4 columns:
- `age_key`: (INT, surrogate PK)
- `age_band`: (STRING) following the age band rules below
- `min_age`: (INT)
- `max_age`: (INT)

Bands:
- "<18": min_age = NULL, max_age = 17
- "18-24": 18, 24
- "25-34": 25, 34
- "35-44": 35, 44
- "45-54": 45, 54
- "55-64": 55, 64
- "65-74": 65, 74
- "75-84": 75, 84
- "85-94": 85, 94
- "unknown": NULL, NULL

The construction of this table is a bit tricky, so we're going to show you how to do it, as follows:

In [ ]:
%%timemem

# Static age bands
age_band_rows = [
    ("<18",   None, 17),
    ("18-24", 18, 24),
    ("25-34", 25, 34),
    ("35-44", 35, 44),
    ("45-54", 45, 54),
    ("55-64", 55, 64),
    ("65-74", 65, 74),
    ("75-84", 75, 84),
    ("85-94", 85, 94),
    ("unknown", None, None),
]
dim_age = spark.createDataFrame(age_band_rows, ["age_band", "min_age", "max_age"])

w_age = Window.orderBy(F.col("age_band"))
dim_age = dim_age.withColumn("age_key", F.dense_rank().over(w_age))

dim_age.count()

**The correct answer should be 10.**

### 4.3 The `brand`, `product`, and `category` Dimension Tables

Build the following dimension tables:

**`dim_brand`:**
- `brand_key` (INT, surrogate PK)
- `brand_code` (STRING) 
- `brand_desc` (STRING)

**`dim_category`:**
- `category_key` (INT, surrogate PK)
- `category_code` (STRING) 
- `category_desc` (STRING)

**`dim_product`:**
- `product_key`  (INT, surrogate PK)
- `product_id`   (STRING)
- `product_desc` (STRING)
- `brand_key`   (INT, FK → `dim_brand`)  
- `category_key`(INT, FK → `dim_category`)

The Learning goals of `dim_product` is to keep all products in `product`, and add details from `product_names`, then join the results with `brand` and `category` dimension tables.

**write some code here**

In [ ]:
%%timemem
# codecell_43k3n9 (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "dim_brand", "dim_category", and "dim_product" should hold 
# the dimension tables according to the specifications above.

print(f"Number of rows in dim_brand: {dim_brand.count()}")
print(f"Number of rows in dim_category: {dim_category.count()}")
print(f"Number of rows in dim_product: {dim_product.count()}")

**Correct answers:**

+ Number of rows in `dim_brand`: 3444
+ Number of rows in `dim_category`: 13
+ Number of rows in `dim_product`: 166794

### 4.4  The `date` Dimension Table

This table is expected to have one row per calendar date. 

**`dim_date`:**
- `date_key`     (INT, surrogate PK; format YYYYMMDD)
- `date`         (DATE, the actual calendar date)
- `day`          (INT, 1–31)
- `day_of_week`  (INT, 1=Mon … 7=Sun)
- `day_name`     (STRING, e.g., Monday)
- `is_weekend`   (BOOLEAN)
- `week_of_year` (INT, 1–53, ISO week)
- `month`        (INT, 1–12)
- `month_name`   (STRING, e.g., January)
- `quarter`      (INT, 1–4)
- `year`         (INT)


There are 2025 years, each with 365 days. Do we need to have a table that big? 
We can, but we do not have to! 

Instead, follow these instructions to create only as many rows as we need:

1. Determine the date range (from the min and max `event_date` in `df_events`).
2. Generate all dates in that range with `F.sequence()`.
3. Derive attributes (`day`, `day_of_week`, ...).
4. Create `date_key` = `year * 10000 + month * 100 + day` (i.e., YYYYMMDD).
5. Assign `date_key` as the surrogate PK.

Build the `dim_date` table conforming to the specifications above.

**write some code here**

In [2]:
%%timemem
# codecell_44qm5c (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "dim_date" should hold the dates dimension table according to the specification above.

print(dim_date.count())

UsageError: Cell magic `%%timemem` not found.


**The correct answer should be 32.**

If you reach here, congratulations!
You have created all the dimension tables!

In [ ]:
%%timemem

print(f"dim_user: {dim_user.count()}")
print(f"dim_age: {dim_age.count()}")
print(f"dim_brand: {dim_brand.count()}")
print(f"dim_category: {dim_category.count()}")
print(f"dim_product: {dim_product.count()}")
print(f"dim_date: {dim_date.count()}")

**Correct answers:**

- `dim_user`: 3022290
- `dim_age`: 10
- `dim_brand`: 3444
- `dim_category`: 13
- `dim_product`: 166794
- `dim_date`: 32

## 5. Build the Fact Table

Now it's time to build the fact table!

Our goal in this step is to create a clean `fact_events` table that joins the events from the operational database to the dimension tables you've just built above.
Along the way, we're going to enforce data quality and do a bit of data cleaning.

### 5.1 Clean Events

Create `events_clean` by removing any record that "does not make sense".
Specifically:

- Start from the `df_events` DataFrame.
- Keep only rows with non-null timestamps, `session_id`, and `product_id`.
- Cast price to double; keep `NULL` prices (views/carts can be price-less) and non-negative values only.
- Drop dates in the future.
- Restrict to valid event types: `view`, `cart`, `purchase`, `remove`.

**write some code here**

In [ ]:
%%timemem
# codecell_51ep7v (keep this id for tracking purposes)

from pyspark.sql import functions as F
from functools import reduce
from operator import and_ as AND

valid_types = ["view", "cart", "purchase", "remove"]

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "events_clean" should conform to the specification above.

events_clean.count()

### 5.2 Cap Silly Prices

Next, let us check some statistics about prices and then decide what we want to do.

What is the minimum, maximum, and average price in this database?

**write some code here**

In [ ]:
%%timemem
# codecell_52hg6x (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "minimum", "maximum", and "average" should conform to the specification above.

print(f"minimum: {minimum}")
print(f"maximum: {maximum}")
print(f"average: {average}")

Wait, something's not right! 
The average price is 864.27 but the maximum seems suss...
It is possible these high prices are just errors.

For simplicity, let us assume a threshold value equal to 100x the average, and remove anything more than that.
Filter `events_clean` as described.

write some code here

In [3]:
%%timemem
# codecell_52bf5d (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "events_clean" should conform to the specification above.

events_clean.count()

UsageError: Cell magic `%%timemem` not found.


Good, we still have about 42.4M records, but we've done some basic data cleaning.
Let us continue...

### 5.3 Build Tiny Lookup Tables (LKPs)

Create lookup tables that help us connect `events_clean` with the dimension tables we created:

- `user_lkp`: (`user_id` → `user_key`) from `dim_user`.
- `prod_lkp`: (`product_id` → `product_key`, `brand_key`, `category_key`) from `dim_product`.
- `date_lkp`: (`date` → `date_key`) from `dim_date`.
- session-to-user bridge: use the raw `df_session` (`session_id`, `user_id`) CSV (not a dimension) to pull `user_id`.

**Hint:** These LKPs are just calling `select` from the right sources with the right parameters.

**write some code here**

In [ ]:
%%timemem
# codecell_53l2kp (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, the following variables should conform to the specification above.

print(session_bridge.count(), user_lkp.count(), prod_lkp.count(), date_lkp.count())

### 5.4 Join Everything Together

Finally, join everything together to create `fact_events`.
Follow the following steps:

- Start from `clean events` with these columns: (`event_time`, `event_type`, `session_id`, `product_id`, `price`, `date`).
- Join sessions first (to get `user_id`).
- Then join product, date, and user.
- Join with `dim_user` to find out the birthdate and compute user age at the day of the event in `age_on_event`.
- Join with `dim_age` to find the age band based on `age_on_event`.

**Hints:**

- You built the LKPs for a reason... use them.
- Left, right, or natural joins?

The final part above is a bit tricky, so we'll just give you the answer. But you'll need to figure out how it integrates with everything above.

```
        .withColumn("age_on_event", F.floor(F.months_between(F.col("date"), F.to_date("birthdate"))/12))
        .join(
           dim_age.select("age_key", "age_band", "min_age", "max_age"),
           (
               ((F.col("age_on_event") > F.col("min_age"))) &
               ((F.col("age_on_event") <= F.col("max_age")))
           ),
           "left"
       )
```

The final result (`fact_events`) should include the following columns:

- `date_key`
- `user_key`
- `age_key`
- `product_key`
- `brand_key`
- `category_key`
- `session_id`
- `event_time`
- `event_type`
- `price`

**write some code here**

In [ ]:
%%timemem
# codecell_54aaaa (keep this id for tracking purposes)

# TODO: Write your code below, but do not remove any lines already in this cell.


# By the time we get to here, "fact_events" should conform to the specification above.

print(fact_events.count())

Congrats, you've done it!
You've created the fact table successfuly! 🚀

Here is the summary of the schema:

- `date_key` (FK → `dim_date`)
- `user_key` (FK → `dim_user`)
- `age_key`  (FK → `dim_age`)
- `product_key` (FK → `dim_product`)
- `brand_key` (FK → `dim_brand`)
- `category_key` (FK → `dim_category`)
- `session_id` (STRING, business key, kept directly in this table)
- `event_time` (TIMESTAMP)
- `event_tpe` (STRING)
- `price` (DOUBLE)


## 6. Export the Fact Table

You now have a shiny `fact_events` table!
But how should you store it?
(Remember our discussion in class about row vs. column representations?)

Let's store `fact_events` in a few different ways and compare data sizes.

First, let's try writing out as CSV files, both compressed and uncompressed, per below.

Note that in Spark, we specify the output _directory_, which is then populated with many "part" files.

In [ ]:
fact_events.write.mode("overwrite").option("header", True).csv(BASE_DIR + "/fact_events.csv")
fact_events.write.mode("overwrite").option("header", True).option("compression", "snappy").csv(BASE_DIR + "/fact_events.csv.snappy")

Let's then try Parquet:

In [ ]:
fact_events.write.mode("overwrite").parquet(BASE_DIR + "/fact_events.parquet")

Let's compare the output sizes using the following bit of code:

In [ ]:
import os
for f in [BASE_DIR + "/fact_events.csv", BASE_DIR + "/fact_events.csv.snappy", BASE_DIR + "/fact_events.parquet"]:
    try:
        size = sum(os.path.getsize(os.path.join(dp, fn))
                   for dp, dn, filenames in os.walk(f)
                   for fn in filenames)
        print(f"{f}: {size/(1024*1024*1024):.1f} GB")
    except FileNotFoundError:
        pass

**your answers below!**

// qcell_6a9876 (keep this id for tracking purposes)

- **Size of CSV output, no compression:** XX  GB
- **Size of CSV output, Snappy compression:** XX GB
- **Size of Parquet output:** XX GB 

**Answer the following question:**

Q6.1 Why is columnar storage (Parquet) usually much smaller?

Q6.2 Which format is better for analytical queries and why?

**your answers below!**

// qcell_6b1234 (keep this id for tracking purposes)

**Q6.1 Answer:**

**your answer!**

**Q6.2 Answer:**

**your answer!**

## 7. Submission

Details about the Submission of this assignment are outlined in the helper.

In [ ]:
%%timemem
spark.stop()

## Deliverables
- This notebook with all code cells executed.
- A brief `REPORT.md` with: inputs, assumptions, plan screenshots, quality results, and performance choices.
- Output folder with Parquet sample (≤20 MB).

## Evaluation
- Correctness and clarity of pipeline (40%).
- Data‑quality gates and rationale (20%).
- Performance reasoning and plan analysis (20%).
- Reproducibility and organization (20%).


## Performance notes
- Record `spark.sql.shuffle.partitions` and justify your value.
- Show one example of avoiding UDFs by using built‑ins.
- If you use broadcast join, explain why it is safe.


## Reproducibility checklist
- List Spark version and key configs.
- Fix time zone to UTC.
- Control randomness if used.
- Provide exact commands to run the notebook end‑to‑end.
